In [1]:
pip install ../.

Processing /home/fkovacev/Uni/PhD/Semantic_Technologies/starvers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for starvers: filename=starvers-0.8.1-py3-none-any.whl size=20097 sha256=60f3278f8c1be1860c19002849a476310bddf2a2d285057782b5c952ce02115e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6fji4r07/wheels/af/a9/8e/8e9f16bbaea5ef5c42307f3009a40ba8bf4043c922a78ca700
Successfully built starvers
  Attempting uninstall: starvers
    Found existing installation: starvers 0.8.1
    Uninstalling starvers-0.8.1:
      Successfully uninstalled starvers-0.8.1
Note: you may need to restart the kernel to use updated packages.


# Set up connection to RDF store

In [2]:
from starvers.starvers import TripleStoreEngine

get_endpoint = "http://ThinkPad-T14s-FK:7200/repositories/hust_demo"
post_endpoint = "http://ThinkPad-T14s-FK:7200/repositories/hust_demo/statements"
engine = TripleStoreEngine(get_endpoint, post_endpoint)

# Clear repository

In [3]:
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST, GET, JSON

sparql_post = SPARQLWrapper(post_endpoint)
sparql_post.setHTTPAuth(DIGEST)
sparql_post.setMethod(POST)

clear = """
clear default
"""
sparql_post.setQuery(clear)
sparql_post.query()

# Load initial dataset

In [4]:
insert = """
insert data {
    <http://example.com/Nguyen_Van_Anh> <http://example.com/occupation> <http://example.com/tour_guide> .
}

"""
sparql_post.setQuery(insert)
sparql_post.query()

# Version all row

In [5]:
import tzlocal
from datetime import datetime, timedelta, timezone
initial_timestamp = datetime(2022, 10, 12, 14, 43, 21, 941000, timezone(timedelta(hours=2)))
engine.version_all_rows(initial_timestamp)

# Insert new triples

In [6]:
new_triples = ['<http://example.com/Le_Van_Thanh> <http://example.com/occupation> <http://example.com/travel_blogger> .',
        '<http://example.com/Tran_Thi_Thu> <http://example.com/occupation> <http://example.com/tour_guide> .']
engine.insert(new_triples)

# Update triples

In [7]:
# <http://example.com/Nguyen_Van_Anh> <http://example.com/occupation> <http://example.com/tour_guide>
# None None <http://example.com/hotel_manager>


engine.update(
old_triples=[['<http://example.com/Nguyen_Van_Anh>','<http://example.com/occupation>', '<http://example.com/tour_guide>']],
new_triples=[[None, None, '<http://example.com/hotel_manager>']])

# Delete triples

In [8]:
# <http://example.com/Nguyen_Van_Anh> <http://example.com/occupation> <http://example.com/hotel_manager>

engine.outdate(['<http://example.com/Nguyen_Van_Anh> <http://example.com/occupation> <http://example.com/hotel_manager> .'])

# Query actual data

In [9]:
query = """
PREFIX vers: <https://github.com/GreenfishK/DataCitation/versioning/>

SELECT ?person ?occupation {
    ?person <http://example.com/occupation> ?occupation .
}
"""

actual_snapshot = engine.query(query)
print(actual_snapshot)

                            person                         occupation
0  http://example.com/Le_Van_Thanh  http://example.com/travel_blogger
1  http://example.com/Tran_Thi_Thu      http://example.com/tour_guide


# Query historical data

In [10]:
snapshot_timestamp = initial_timestamp
historical_snapshot = engine.query(query, snapshot_timestamp)
print(historical_snapshot)

                              person                     occupation
0  http://example.com/Nguyen_Van_Anh  http://example.com/tour_guide


# Timestamped SPARQL-star query 

In [11]:
from starvers.starvers import timestamp_query
timestamped_query, timestamp = timestamp_query(query)
print(timestamped_query)


PREFIX vers: <https://github.com/GreenfishK/DataCitation/versioning/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?person ?occupation{<< <<?person <http://example.com/occupation> ?occupation>> vers:valid_from ?valid_from_1 >> vers:valid_until ?valid_until_1.
filter(?valid_from_1 <= ?tsBGP_0 && ?tsBGP_0 < ?valid_until_1)

bind("2023-03-09T10:36:40.638+00:00"^^xsd:dateTime as ?tsBGP_0)}


In [12]:
print(timestamp)

2023-03-09T10:36:40.638+00:00
